In [1]:
import MDAnalysis as mda
import numpy as np
import os
from utils import *
import warnings 

# Suppress warnings specific to MDAnalysis
warnings.filterwarnings("ignore", category=UserWarning, module="MDAnalysis")

c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
residue_charge_dict = {'ARG':1,'LYS':1,'ASP':-1,'GLU':-1,'INI':-5}

In [3]:
complex = mda.Universe('complexes/6/qm_complex.pdb')
ini = complex.select_atoms("resname INI")

In [4]:
aka_atom_dict = get_substrate_aka_indexes(ini)
C2_index = aka_atom_dict['C2']
C2_id = ini.atoms[C2_index].index
C2_atom = complex.select_atoms("index " +  str(C2_id))

In [5]:
dist = 6
# get atoms a certain distance away from the carbonyl carbon  
QM_shell= complex.select_atoms("around " + str(dist) + " index " + str(C2_id))
# get the residues those atoms belong to
QM_residues = set([atom.residue for atom in QM_shell])
# do the same for the MM atoms expanding our shell 
active_atoms = complex.select_atoms("around " + str(dist) + " index " + str(C2_id))
active_residues = set([atom.residue for atom in active_atoms])
# do the same for the fixed atoms that are used in the optimizer
fixed_shell = complex.select_atoms("around " + str(dist+2) + " index " + str(C2_id))
QMMM_plus_fixed_residues = set([atom.residue for atom in fixed_shell])

# identify just the fixed atoms 
fixed_residues = QMMM_plus_fixed_residues.difference(active_residues)

# get all the atoms that belong to the residues for each of these groups 
QM_atoms = complex.select_atoms(" or ".join([f"resid {residue.resid}" for residue in QM_residues]))
active_atoms = complex.select_atoms(" or ".join([f"resid {residue.resid}" for residue in active_residues]))
fixed_atoms = complex.select_atoms(" or ".join([f"resid {residue.resid}" for residue in fixed_residues]))

In [6]:
total_charge = 0
# get charge of QM region 
for residue in list(QM_residues):
    resname = residue.resname
    if resname in residue_charge_dict:
        total_charge += residue_charge_dict[resname]

print(total_charge)

-3


In [7]:
def simplify_integer_list(int_list):
    if not int_list:
        return ""

    # Sort the list to make sure the integers are in ascending order
    sorted_list = sorted(int_list)
    ranges = []
    start = sorted_list[0]
    end = start

    for number in sorted_list[1:]:
        if number == end + 1:
            end = number
        else:
            if start == end:
                ranges.append(f"{start}")
            else:
                ranges.append(f"{start}:{end}")
            start = number
            end = start

    # Add the last range or number
    if start == end:
        ranges.append(f"{start}")
    else:
        ranges.append(f"{start}:{end}")

    return ' '.join(ranges)

In [8]:
QM_atom_indexes = [atom.index for atom in QM_atoms]
active_atom_indexes = [atom.index for atom in active_atoms]
fixed_atom_indexes = [atom.index for atom in fixed_atoms]

In [9]:
QM_list = simplify_integer_list(QM_atom_indexes)
active_list = simplify_integer_list(active_atom_indexes)
fixed_list = simplify_integer_list(fixed_atom_indexes)

In [14]:
# Define the input and output file paths
input_file = "template_QMMM_script.inp"
output_file = "complexes/6/opt.inp"

# Open the input file and read its contents
with open(input_file, 'r') as file:
    content = file.read()

# Replace the {} placeholders with the variable values
content = content.replace("{}", "{" + QM_list + "}" , 1)  # First occurrence
content = content.replace("{}", "{" + active_list + "}", 1)  # Second occurrence
content = content.replace("{}", "{" + fixed_list + "}", 1)  # Third occurrence

# Add the custom line to the end
custom_line = "*pdbfile " + str(total_charge) +" 1 qm_complex.pdb\n"
content += custom_line

# Write the modified content to the output file
with open(output_file, 'w') as file:
    file.write(content)

print(f"File processed and saved as {output_file}")


File processed and saved as complexes/6/opt.inp
